<a href="https://colab.research.google.com/github/ronron-gh/ESP32_EdgeAI_PlatformIO/blob/main/for_moduleLLM/nnabla_convert_mydata_model_to_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nnablaをインストールする。

In [ ]:
!pip install nnabla-ext-cuda116

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


こちらのトラブルシュートに従い、CUDAのツールキットを追加でインストールする  
https://github.com/ai-forever/ghost/issues/87

In [ ]:
!apt install nvidia-cuda-toolkit --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

インストールできたか確認。以下が出ればOK。  
Initializing CPU extension...  
Initializing CUDA extension...  
Initializing cuDNN extension...  


In [ ]:
!python -c "import nnabla_ext.cuda, nnabla_ext.cudnn"

2024-12-29 02:25:28,973 [nnabla][INFO]: Initializing CPU extension...
2024-12-29 02:25:30,250 [nnabla][INFO]: Initializing CUDA extension...
2024-12-29 02:25:30,323 [nnabla][INFO]: Initializing cuDNN extension...


nnablaのサンプル群をGitHubから取得。

In [ ]:
!git clone https://github.com/sony/nnabla-examples.git

Cloning into 'nnabla-examples'...
remote: Enumerating objects: 9648, done.
remote: Counting objects: 100% (2206/2206), done.
remote: Compressing objects: 100% (916/916), done.
remote: Total 9648 (delta 1291), reused 2077 (delta 1230), pack-reused 7442 (from 1)
Receiving objects: 100% (9648/9648), 298.96 MiB | 34.66 MiB/s, done.
Resolving deltas: 100% (5255/5255), done.
Updating files: 100% (1711/1711), done.


本ブログで公開しているデータセットやプログラムをGitHubから取得。

In [ ]:
!git clone https://github.com/ronron-gh/ESP32_EdgeAI_PlatformIO.git

Cloning into 'ESP32_EdgeAI_PlatformIO'...
remote: Enumerating objects: 751, done.
remote: Counting objects: 100% (751/751), done.
remote: Compressing objects: 100% (676/676), done.
remote: Total 751 (delta 72), reused 740 (delta 68), pack-reused 0 (from 0)
Receiving objects: 100% (751/751), 2.46 MiB | 29.64 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [ ]:
%cd ESP32_EdgeAI_PlatformIO

/content/ESP32_EdgeAI_PlatformIO


データセットをnnablaに読み込ませるためのリストファイル(train.csv, test.csv)を作成する。画像は28x28のモノクロに変換する。（CSVの作成と画像の変換を行うPythonスクリプトを用意しました。）

In [ ]:
!python make_dataset_csv.py my_dataset/finger_direction

The current folder is my_dataset/finger_direction
The current folder is my_dataset/finger_direction/3
listed file : my_dataset/finger_direction/3/0014.jpg
listed file : my_dataset/finger_direction/3/0027.jpg
listed file : my_dataset/finger_direction/3/0032.jpg
listed file : my_dataset/finger_direction/3/0025.jpg
listed file : my_dataset/finger_direction/3/0035.jpg
listed file : my_dataset/finger_direction/3/0060.jpg
listed file : my_dataset/finger_direction/3/0047.jpg
listed file : my_dataset/finger_direction/3/0069.jpg
listed file : my_dataset/finger_direction/3/0036.jpg
listed file : my_dataset/finger_direction/3/0011.jpg
listed file : my_dataset/finger_direction/3/0033.jpg
listed file : my_dataset/finger_direction/3/0064.jpg
listed file : my_dataset/finger_direction/3/0034.jpg
listed file : my_dataset/finger_direction/3/0055.jpg
listed file : my_dataset/finger_direction/3/0067.jpg
listed file : my_dataset/finger_direction/3/0063.jpg
listed file : my_dataset/finger_direction/3/0040.j

train.csv, test.csv, converted_datasetsフォルダが作成されていることを確認する。

In [ ]:
!ls

converted_datasets   my_dataset					nnabla-example-modify  test.csv
LICENSE		     nnabla_convert_mnist_model_to_csrc.ipynb	Platformio	       train.csv
make_dataset_csv.py  nnabla_convert_mydata_model_to_csrc.ipynb	README.md


nnablaのMNISTサンプルコードのディレクトリに、作成したCSV、データセット（変換後）、改造したプログラムをコピー。

In [ ]:
!cp *.csv ../nnabla-examples/image-classification/mnist-collection/

In [ ]:
!cp -r converted_datasets ../nnabla-examples/image-classification/mnist-collection/

In [ ]:
!cp nnabla-example-modify/classification_mydata.py ../nnabla-examples/image-classification/mnist-collection/

mnistサンプルプログラムのフォルダに移動し、学習を実行。

In [ ]:
%cd ../nnabla-examples/image-classification/mnist-collection/

/content/nnabla-examples/image-classification/mnist-collection


In [ ]:
!python classification_mydata.py -c cudnn -n lenet -o output

2024-12-29 02:29:22,095 [nnabla][INFO]: Initializing CPU extension...
2024-12-29 02:29:23,122 [nnabla][INFO]: Running in cudnn
2024-12-29 02:29:23,359 [nnabla][INFO]: Initializing CUDA extension...
2024-12-29 02:29:23,395 [nnabla][INFO]: Initializing cuDNN extension...
2024-12-29 02:29:26,503 [nnabla][INFO]: Saving output/lenet_result_epoch0.nnp as nnp
2024-12-29 02:29:26,503 [nnabla][INFO]: Saving <_io.StringIO object at 0x79969834e050> as prototxt
2024-12-29 02:29:26,510 [nnabla][INFO]: Parameter save (.h5): <_io.BytesIO object at 0x7996983b1080>
2024-12-29 02:29:26,511 [nnabla][INFO]: Model file is saved as (.nnp): output/lenet_result_epoch0.nnp
2024-12-29 02:29:26,511 [nnabla][INFO]: DataSource with shuffle(True)
2024-12-29 02:29:26,514 [nnabla][INFO]: Using DataSourceWithFileCache
2024-12-29 02:29:26,514 [nnabla][INFO]: DataSource with shuffle(True)
2024-12-29 02:29:26,514 [nnabla][INFO]: Cache Directory is None
2024-12-29 02:29:26,514 [nnabla][INFO]: Cache size is 100
2024-12-29 

outputフォルダにモデルファイルnnpが作られていることを確認。

In [ ]:
!ls output

checkpoint_0.json     checkpoint_7000.json     params_1000.h5  params_8000.h5  states_5000.h5
checkpoint_1000.json  checkpoint_8000.json     params_2000.h5  params_9000.h5  states_6000.h5
checkpoint_2000.json  checkpoint_9000.json     params_3000.h5  states_0.h5     states_7000.h5
checkpoint_3000.json  lenet_params_010000.h5   params_4000.h5  states_1000.h5  states_8000.h5
checkpoint_4000.json  lenet_result_epoch0.nnp  params_5000.h5  states_2000.h5  states_9000.h5
checkpoint_5000.json  lenet_result.nnp	       params_6000.h5  states_3000.h5
checkpoint_6000.json  params_0.h5	       params_7000.h5  states_4000.h5


nnpをonnxに変換するためのパッケージをインストール

In [ ]:
!pip install nnabla_converter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6

nnpをonnxに変換（何やらWarningが出てくるが、onnxが出力されていればOK）。

In [ ]:
!nnabla_cli convert ./output/lenet_result.nnp ./output/lenet_result.onnx

2024-12-29 02:46:58,856 [nnabla][INFO]: Initializing CPU extension...
NNabla command line interface (Version:1.39.0, Build:240523014612)
2024-12-29 02:46:59.830236: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-29 02:46:59.830287: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-29 02:46:59.830328: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-29 02:47:00.963159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-29 02:47:02,145 [numexpr.utils][INFO]: NumExpr defaulting to 2 threads.
/usr/local/lib/python

In [ ]:
!ls output

checkpoint_0.json     checkpoint_7000.json     params_0.h5     params_7000.h5  states_4000.h5
checkpoint_1000.json  checkpoint_8000.json     params_1000.h5  params_8000.h5  states_5000.h5
checkpoint_2000.json  checkpoint_9000.json     params_2000.h5  params_9000.h5  states_6000.h5
checkpoint_3000.json  lenet_params_010000.h5   params_3000.h5  states_0.h5     states_7000.h5
checkpoint_4000.json  lenet_result_epoch0.nnp  params_4000.h5  states_1000.h5  states_8000.h5
checkpoint_5000.json  lenet_result.nnp	       params_5000.h5  states_2000.h5  states_9000.h5
checkpoint_6000.json  lenet_result.onnx        params_6000.h5  states_3000.h5


変換後のファイルをPCにダウンロードするために、Google Driveに一旦コピーする。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ./output/lenet_result.onnx /content/drive/MyDrive/